In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [10]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data & set config

In [11]:
config = get_config(nb_name)

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [13]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [14]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [15]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [16]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [17]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     85   10    9  112.90  555  112.83  0.06  0.00  105.35  530  105.33   
10000    65   20   19  112.38  534  112.11  0.27  0.00  104.50  485  103.73   
15000    77   30   29  111.88  513  111.36  0.52  0.00  104.34  472  103.30   
20000    62   40   39  111.55  498  110.82  0.73  0.00  104.05  456  102.61   
25000    80   50   49  111.27  485  110.33  0.93  0.01  103.82  444  102.17   
30000    74   60   59  111.06  476  109.97  1.08  0.00  103.80  443  102.13   
35000    81   70   69  110.89  468  109.68  1.21  0.00  103.70  435  101.84   
40000    73   80   79  110.76  462  109.45  1.30  0.00  103.59  433  101.75   
45000    81   90   89  110.65  457  109.25  1.39  0.00  103.52  428  101.56   
50000    75  100   99  110.56  453  109.10  1.46  0.00  103.48  427  101.56   
55000   106  110  109  110.48  449  108.95  1.52  0.00  103.41  423  101.38   
60000    83  120  119  110.41  447  108.84  1.57  0.00  103.48  425  101.52   
65000    82  130  129  110.36  444  108.73  1.62  0.00  103.40  421  101.33   
70000    73  140  139  110.31  442  108.65  1.66  0.00  103.42  422  101.40   
75000    81  150  149  110.26  440  108.56  1.70  0.00  103.39  420  101.30   
80000    73  160  159  110.23  438  108.50  1.73  0.00  103.38  421  101.33   
85000    82  170  169  110.19  436  108.43  1.76  0.00  103.34  418  101.21   
90000    79  180  179  110.16  435  108.37  1.78  0.00  103.38  421  101.31   
95000    82  190  189  110.13  433  108.32  1.81  0.00  103.56  425  101.43   
100000   81  200  199  110.10  432  108.26  1.83  0.00  103.48  422  101.32   
105000   84  210  209  110.08  431  108.22  1.85  0.00  103.56  426  101.49   
110000   91  220  219  110.06  430  108.18  1.87  0.00  103.43  420  101.25   
115000   84  230  229  110.04  429  108.14  1.89  0.00  103.45  422  101.35   
120000   86  240  239  110.02  428  108.11  1.91  0.00  103.42  419  101.23   
125000   84  250  249  110.00  427  108.08  1.92  0.00  103.41  420  101.30   
130000  100  260  259  109.98  427  108.04  1.94  0.00  103.34  416  101.15   
135000  113  270  269  109.97  426  108.01  1.95  0.00  103.38  419  101.28   
140000  116  280  279  109.95  425  107.98  1.96  0.00  103.33  416  101.12   
145000  102  290  289  109.94  425  107.96  1.97  0.00  103.35  419  101.23   
150000  113  300  299  109.92  424  107.93  1.99  0.00  103.31  416  101.11   
155000   84  310  309  109.91  423  107.91  2.00  0.00  103.34  418  101.21   
160000   85  320  319  109.90  423  107.89  2.01  0.00  103.31  415  101.09   
165000   83  330  329  109.89  422  107.87  2.02  0.00  103.34  417  101.18   
170000   88  340  339  109.88  422  107.85  2.03  0.00  103.32  415  101.08   

                     TEST:      SPEC:             HIER:        
          KL   REG    LOSS  PPL     1     2     3 CHILD OTHER  
5000    0.01  0.00  103.48  528  0.05  0.09  0.13  0.97  0.96  
10000   0.77  0.00  102.58  478  0.16  0.39  0.40  0.38  0.35  
15000   1.04  0.00  102.35  463  0.18  0.62  0.40  0.19  0.22  
20000   1.44  0.00  101.83  440  0.19  0.55  0.46  0.33  0.18  
25000   1.64  0.00  101.56  428  0.22  0.52  0.48  0.45  0.16  
30000   1.67  0.00  101.64  429  0.22  0.57  0.49  0.37  0.15  
35000   1.86  0.00  101.52  421  0.26  0.58  0.48  0.44  0.12  
40000   1.84  0.00  101.46  420  0.24  0.59  0.49  0.34  0.12  
45000   1.96  0.00  101.36  415  0.27  0.57  0.47  0.43  0.12  
50000   1.92  0.00  101.35  415  0.24  0.59  0.50  0.32  0.12  
55000   2.03  0.00  101.25  410  0.28  0.57  0.48  0.43  0.12  
60000   1.95  0.00  101.25  410  0.24  0.61  0.49  0.29  0.13  
65000   2.07  0.00  101.28  409  0.27  0.56  0.48  0.42  0.13  
70000   2.02  0.00  101.28  409  0.25  0.61  0.50  0.28  0.12  
75000   2.09  0.00  101.26  409  0.27  0.56  0.49  0.43  0.13  
80000   2.05  0.00  101.26  409  0.26  0.60  0.51  0.27  0.12  
85000   2.13  0.00  101.23

0 R: 1.000 P: 0.299 cover ! color love keyboard pro bottom mac nice perfectly
   1 R: 0.282 P: 0.116 sleeve protection inside inch air nice zipper smell material pro
     11 R: 0.061 P: 0.061 months broke started quality return year bought weeks time month
     14 R: 0.079 P: 0.079 strap - : handle shoulder zipper top side zippers straps
     13 R: 0.027 P: 0.027 quality price 'm padding inside thing leather 've inch nice
   2 R: 0.310 P: 0.104 pocket mouse netbook power ipad room charger cord perfectly perfect
     24 R: 0.085 P: 0.085 carry work back lot heavy books pack bought stuff comfortable
     21 R: 0.070 P: 0.070 pockets pocket compartment small strap shoulder carry hold large items
     23 R: 0.051 P: 0.051 room carry perfect love plenty ipad cords recommend ! accessories
   4 R: 0.049 P: 0.013 ! love perfect perfectly ... ; pro & price hp
     41 R: 0.036 P: 0.036 ! love ... bought loves gift price perfect amazon quality
   3 R: 0.059 P: 0.017 & ; pro size big air descripti

KeyboardInterrupt: 